<a href="https://colab.research.google.com/github/MikolajWasowski/neural-network-course/blob/master/03_keras/01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Konfiguracja środowiska


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

'2.15.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [2]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [3]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
Total params: 44 (176.00 Byte)
Trainable params: 44 (176.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Dodanie kolejnej warstwy

In [5]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
Total params: 54 (216.00 Byte)
Trainable params: 54 (216.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [6]:
from tensorflow.keras.activations import linear

random_data = sorted(np.random.randn(200))
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-2.830073,-2.830073
1,-2.261445,-2.261445
2,-2.220692,-2.220692
3,-2.143522,-2.143522
4,-2.131811,-2.131811


In [7]:
px.line(data, x='data', y='linear', width=800, range_y=[-2, 2])

In [8]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-2.830073,0.055721
1,-2.261445,0.094367
2,-2.220692,0.097908
3,-2.143522,0.104938
4,-2.131811,0.106043


In [9]:
px.line(data, x='data', y='sigmoid', width=800, range_y=[-0.5, 1.5])

In [10]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-2.830073,0.0
1,-2.261445,0.0
2,-2.220692,0.0
3,-2.143522,0.0
4,-2.131811,0.0


In [11]:
px.line(data, x='data', y='relu', width=800, range_y=[-0.5, 1.5])

In [12]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-2.830073,-0.993060
1,-2.261445,-0.978518
2,-2.220692,-0.976715
3,-2.143522,-0.972882
4,-2.131811,-0.972248


In [13]:
px.line(data, x='data', y='tanh', width=800, range_y=[-1.5, 1.5])

In [14]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/))
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [15]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [ ]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [16]:
data = np.random.randn(1000, 150)
labels = np.random.randint(2, size=(1000, 1))

print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [17]:
data[:3]

array([[-7.39868959e-01, -3.12413951e-01,  2.34894814e-01,
         1.56744605e+00, -1.08816955e+00, -1.22212086e+00,
        -8.25006605e-01,  1.68368956e-01,  4.39138078e-01,
        -6.06538777e-01, -1.73063175e+00, -2.36286612e-01,
         1.19239499e-01,  1.17515006e+00,  3.09100347e-01,
        -1.03742185e-01,  1.02167191e+00, -8.73571459e-02,
         1.98458247e-01,  5.36982587e-01, -4.58724800e-01,
        -7.06952735e-02, -3.64224062e-01,  3.45863691e-01,
        -8.67942830e-01,  2.27843176e-01, -3.44136470e-01,
        -1.15191230e+00, -1.98556893e+00, -1.32004052e+00,
         7.16266623e-01,  8.46389949e-01,  4.74417294e-01,
        -1.84894114e+00,  8.67197335e-01,  1.60952737e+00,
         7.80329693e-01, -1.73824930e-01, -1.06838512e+00,
         9.87723537e-01,  8.49247761e-01,  1.52752491e+00,
        -7.98626170e-01, -4.59485538e-01,  6.75516952e-01,
        -6.38088435e-01, -1.65573421e-01,  1.85843357e+00,
        -9.55959359e-02, -1.78624121e+00, -1.36807421e+0

In [18]:
labels[:10]

array([[1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1]])

In [19]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Epoch 1/20
32/32 [==============================] - 1s 3ms/step - loss: 0.9226 - accuracy: 0.5020
Epoch 2/20
32/32 [==============================] - 0s 2ms/step - loss: 0.7465 - accuracy: 0.5680
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6662 - accuracy: 0.6220
Epoch 4/20
32/32 [==============================] - 0s 3ms/step - loss: 0.6122 - accuracy: 0.6810
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5723 - accuracy: 0.7150
Epoch 6/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5373 - accuracy: 0.7390
Epoch 7/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5072 - accuracy: 0.7670
Epoch 8/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4795 - accuracy: 0.7970
Epoch 9/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4545 - accuracy: 0.8160
Epoch 10/20
32/32 [==============================] - 0s 3ms/step - loss: 0.4302 - accuracy: 0.8390
Epoch 11/20
32/32 [

In [20]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=30)

Epoch 1/20
34/34 [==============================] - 1s 4ms/step - loss: 0.8234 - accuracy: 0.4820
Epoch 2/20
34/34 [==============================] - 0s 3ms/step - loss: 0.7288 - accuracy: 0.5320
Epoch 3/20
34/34 [==============================] - 0s 3ms/step - loss: 0.6744 - accuracy: 0.5860
Epoch 4/20
34/34 [==============================] - 0s 4ms/step - loss: 0.6332 - accuracy: 0.6420
Epoch 5/20
34/34 [==============================] - 0s 3ms/step - loss: 0.5990 - accuracy: 0.6630
Epoch 6/20
34/34 [==============================] - 0s 3ms/step - loss: 0.5677 - accuracy: 0.7080
Epoch 7/20
34/34 [==============================] - 0s 3ms/step - loss: 0.5396 - accuracy: 0.7520
Epoch 8/20
34/34 [==============================] - 0s 3ms/step - loss: 0.5145 - accuracy: 0.7650
Epoch 9/20
34/34 [==============================] - 0s 3ms/step - loss: 0.4907 - accuracy: 0.7780
Epoch 10/20
34/34 [==============================] - 0s 3ms/step - loss: 0.4675 - accuracy: 0.8020
Epoch 11/20
34/34 [

In [21]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
25/25 [==============================] - 1s 11ms/step - loss: 0.8021 - accuracy: 0.4988 - val_loss: 0.7758 - val_accuracy: 0.4950
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 0.7065 - accuracy: 0.5638 - val_loss: 0.7645 - val_accuracy: 0.4900
Epoch 3/20
25/25 [==============================] - 0s 4ms/step - loss: 0.6536 - accuracy: 0.6125 - val_loss: 0.7613 - val_accuracy: 0.4750
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6115 - accuracy: 0.6538 - val_loss: 0.7617 - val_accuracy: 0.4700
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5760 - accuracy: 0.7063 - val_loss: 0.7649 - val_accuracy: 0.4700
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5442 - accuracy: 0.7412 - val_loss: 0.7710 - val_accuracy: 0.4850
Epoch 7/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5151 - accuracy: 0.7725 - val_loss: 0.7791 - val_accuracy: 0.4800
Epoch 8/20
25/25 [=

In [22]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [23]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [24]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [25]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
25/25 [==============================] - 1s 10ms/step - loss: 0.7877 - accuracy: 0.5113 - val_loss: 0.8018 - val_accuracy: 0.5200
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6956 - accuracy: 0.5650 - val_loss: 0.7821 - val_accuracy: 0.5150
Epoch 3/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6435 - accuracy: 0.6325 - val_loss: 0.7707 - val_accuracy: 0.5100
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6017 - accuracy: 0.6787 - val_loss: 0.7645 - val_accuracy: 0.5200
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5661 - accuracy: 0.7212 - val_loss: 0.7614 - val_accuracy: 0.5150
Epoch 6/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5345 - accuracy: 0.7550 - val_loss: 0.7629 - val_accuracy: 0.5100
Epoch 7/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5059 - accuracy: 0.7850 - val_loss: 0.7662 - val_accuracy: 0.5000
Epoch 8/20
25/25 [=

In [26]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [27]:
model.predict(test_data)

1/1 [==============================] - 0s 92ms/step


array([[0.13844213],
       [0.01177152],
       [0.82608587],
       [0.8736388 ],
       [0.30251023]], dtype=float32)

In [30]:
# model.predict_classes(test_data)
np.argmax(model.predict([test_data]),axis=1)

1/1 [==============================] - 0s 27ms/step


array([0, 0, 0, 0, 0])

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [31]:
data = np.random.random((1000, 150))
labels = np.random.randint(10, size=(1000, 1))

In [32]:
print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [33]:
data[:3]

array([[0.87059882, 0.18526764, 0.62655261, 0.24254654, 0.39698467,
        0.2083173 , 0.36746478, 0.54267581, 0.18665382, 0.52259485,
        0.76532271, 0.22034749, 0.59892326, 0.33704254, 0.29890304,
        0.5764454 , 0.33051808, 0.89748431, 0.26548045, 0.56633716,
        0.19612378, 0.26875532, 0.65953067, 0.54493049, 0.47099873,
        0.21745652, 0.35748556, 0.60944563, 0.19964613, 0.22024367,
        0.70092363, 0.57789448, 0.45381274, 0.93655545, 0.47031412,
        0.88579751, 0.46024371, 0.91015562, 0.51591922, 0.66208445,
        0.92901845, 0.64967254, 0.48422373, 0.06034776, 0.94281289,
        0.73987612, 0.77062622, 0.33369539, 0.80749066, 0.10670259,
        0.42405703, 0.19262626, 0.09719076, 0.24770021, 0.84725227,
        0.42993659, 0.76006786, 0.82505741, 0.17538209, 0.54280804,
        0.59523593, 0.0756474 , 0.73979277, 0.13465165, 0.24407931,
        0.09250047, 0.82159955, 0.91336674, 0.77325153, 0.41897029,
        0.4339543 , 0.30141416, 0.59221951, 0.79

In [34]:
labels[:10]

array([[3],
       [5],
       [0],
       [6],
       [7],
       [9],
       [6],
       [0],
       [9],
       [6]])

In [35]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [36]:
labels[1]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [37]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 11ms/step - loss: 2.3886 - accuracy: 0.0850 - val_loss: 2.3789 - val_accuracy: 0.1150
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 2.3355 - accuracy: 0.1000 - val_loss: 2.3836 - val_accuracy: 0.1100
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 2.3098 - accuracy: 0.1175 - val_loss: 2.3875 - val_accuracy: 0.0950
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2925 - accuracy: 0.1200 - val_loss: 2.3940 - val_accuracy: 0.1000
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2769 - accuracy: 0.1412 - val_loss: 2.4027 - val_accuracy: 0.0850
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2625 - accuracy: 0.1312 - val_loss: 2.4001 - val_accuracy: 0.0800
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2466 - accuracy: 0.1625 - val_loss: 2.3873 - val_accuracy: 0.0800
Epoch 8/30
25/25 [=

In [38]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 65ms/step


array([[0.034423  , 0.19718066, 0.02782645, 0.10375426, 0.05479557,
        0.10958991, 0.08476692, 0.05603995, 0.25238806, 0.07923514],
       [0.06338953, 0.12050509, 0.05425838, 0.0673142 , 0.07621435,
        0.08992819, 0.2782857 , 0.06630394, 0.1562993 , 0.02750133],
       [0.05700229, 0.1644979 , 0.04569856, 0.03740943, 0.03914012,
        0.15948883, 0.04979756, 0.18156423, 0.1286781 , 0.13672301],
       [0.02891554, 0.22784333, 0.05868195, 0.04762591, 0.02228471,
        0.09557649, 0.06280507, 0.20774649, 0.16907524, 0.07944521],
       [0.03411413, 0.08796184, 0.04910204, 0.1312851 , 0.02078518,
        0.18763824, 0.05405382, 0.3276781 , 0.04179151, 0.0655901 ],
       [0.09000365, 0.14075032, 0.1498025 , 0.08498677, 0.05459753,
        0.13173008, 0.06813964, 0.08253836, 0.16117826, 0.03627285],
       [0.08761305, 0.21909505, 0.04181908, 0.02528271, 0.0135838 ,
        0.1605419 , 0.08622306, 0.21793467, 0.04750093, 0.10040581],
       [0.12954499, 0.191891  , 0.0894362

In [39]:
np.argmax(model.predict([test_data]),axis=1)

1/1 [==============================] - 0s 55ms/step


array([8, 6, 7, 1, 7, 8, 1, 1, 9, 6])

### <a name='a6'></a> Przykład - regresja

In [40]:
data = np.random.random((1000, 150))
labels = 50 * np.random.random(1000)

In [41]:
data[:3]

array([[0.43397094, 0.4421883 , 0.89371478, 0.71394469, 0.66764667,
        0.32047434, 0.85167535, 0.40884153, 0.24471669, 0.96528199,
        0.94995253, 0.69350052, 0.84048874, 0.71622756, 0.52851942,
        0.19358404, 0.19392345, 0.64826157, 0.72871542, 0.43651476,
        0.84852477, 0.85540207, 0.95239853, 0.9582112 , 0.56735504,
        0.18692151, 0.17620789, 0.49592157, 0.3119545 , 0.11535785,
        0.37429963, 0.34511782, 0.06244098, 0.15638146, 0.02446566,
        0.66286253, 0.98974502, 0.59456814, 0.16919683, 0.05272636,
        0.53516802, 0.58589251, 0.13111043, 0.6427448 , 0.2882978 ,
        0.99796352, 0.86872875, 0.65900401, 0.32499126, 0.14799482,
        0.23500171, 0.3550156 , 0.29599146, 0.71469721, 0.95390892,
        0.03011662, 0.93488357, 0.93977155, 0.52099488, 0.52818759,
        0.11933072, 0.04080208, 0.53728012, 0.94547635, 0.44169857,
        0.60810411, 0.93300083, 0.32197077, 0.27438265, 0.92321994,
        0.8553804 , 0.76879964, 0.47203161, 0.03

In [42]:
labels[:10]

array([46.03725189, 14.17700972, 23.72676303,  4.18379694, 41.44210331,
       16.83861338,  0.7721302 , 22.35169655,  8.49587438,  2.96544863])

In [43]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 8ms/step - loss: 527.3259 - val_loss: 342.5654
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 267.5217 - val_loss: 210.2194
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 205.3557 - val_loss: 200.2827
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 202.6317 - val_loss: 200.2581
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 202.3902 - val_loss: 199.6437
Epoch 6/30
25/25 [==============================] - 0s 3ms/step - loss: 202.2240 - val_loss: 199.6629
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 200.9842 - val_loss: 199.9751
Epoch 8/30
25/25 [==============================] - 0s 3ms/step - loss: 200.6561 - val_loss: 199.7587
Epoch 9/30
25/25 [==============================] - 0s 4ms/step - loss: 200.0852 - val_loss: 200.6038
Epoch 10/30
25/25 [==============================] - 0s 3ms/step - loss: 199.4227 

In [44]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 10ms/step - loss: 22.1055 - mse: 680.5535 - val_loss: 19.1588 - val_mse: 527.9791
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 16.8877 - mse: 432.3296 - val_loss: 15.0180 - val_mse: 334.4890
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 13.8521 - mse: 283.0201 - val_loss: 12.8673 - val_mse: 232.4642
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 12.6268 - mse: 217.5674 - val_loss: 12.0920 - val_mse: 204.2663
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 12.4383 - mse: 204.0243 - val_loss: 11.9746 - val_mse: 200.9927
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 12.4082 - mse: 202.0670 - val_loss: 11.9605 - val_mse: 200.5426
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 12.4003 - mse: 202.2694 - val_loss: 11.9630 - val_mse: 200.5944
Epoch 8/30
25/25 [=============================

In [46]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 21ms/step


array([[26.954998],
       [24.652166],
       [25.66503 ],
       [24.35692 ],
       [25.317545],
       [22.911661],
       [23.484037],
       [28.524988],
       [26.174173],
       [25.29324 ]], dtype=float32)